In [27]:
from collections import Counter

import plotly.express as px
import pyfaidx 
import pandas as pd

In [14]:
genes = pyfaidx.Fasta("../data/IIPR002547.genes.fasta")
genes = [gene for gene in genes if set(str(gene)).issubset({"A", "G", "C", "T"})]

In [19]:
px.violin([len(gene) for gene in genes], title="Length distribution")

In [16]:
Counter(''.join(str(gene) for gene in genes))

Counter({'G': 4265389, 'A': 4167403, 'C': 3819025, 'T': 3539282})

In [34]:
def kmerize(sequence: str, k=3):
    return [sequence[i:i+k] for i in range(len(sequence) - k)]

df = pd.DataFrame.from_dict(dict(Counter(kmer for gene in genes for kmer in kmerize(str(gene)))), orient="index")
px.violin(data_frame=df, hover_name=df.index, points="all")